# APPROXIMATE MATCHING

In [3]:
import string

def z_array(s):
    """ Use Z algorithm (Gusfield theorem 1.4.1) to preprocess s """
    assert len(s) > 1
    z = [len(s)] + [0] * (len(s)-1)
    # Initial comparison of s[1:] with prefix
    for i in range(1, len(s)):
        if s[i] == s[i-1]:
            z[1] += 1
        else:
            break
    r, l = 0, 0
    if z[1] > 0:
        r, l = z[1], 1
    for k in range(2, len(s)):
        assert z[k] == 0
        if k > r:
            # Case 1
            for i in range(k, len(s)):
                if s[i] == s[i-k]:
                    z[k] += 1
                else:
                    break
            r, l = k + z[k] - 1, k
        else:
            # Case 2
            # Calculate length of beta
            nbeta = r - k + 1
            zkp = z[k - l]
            if nbeta > zkp:
                # Case 2a: Zkp wins
                z[k] = zkp
            else:
                # Case 2b: Compare characters just past r
                nmatch = 0
                for i in range(r+1, len(s)):
                    if s[i] == s[i - k]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                z[k] = r - k + 1
    return z


def n_array(s):
    """ Compile the N array (Gusfield theorem 2.2.2) from the Z array """
    return z_array(s[::-1])[::-1]


def big_l_prime_array(p, n):
    """ Compile L' array (Gusfield theorem 2.2.2) using p and N array.
        L'[i] = largest index j less than n such that N[j] = |P[i:]| """
    lp = [0] * len(p)
    for j in range(len(p)-1):
        i = len(p) - n[j]
        if i < len(p):
            lp[i] = j + 1
    return lp


def big_l_array(p, lp):
    """ Compile L array (Gusfield theorem 2.2.2) using p and L' array.
        L[i] = largest index j less than n such that N[j] >= |P[i:]| """
    l = [0] * len(p)
    l[1] = lp[1]
    for i in range(2, len(p)):
        l[i] = max(l[i-1], lp[i])
    return l


def small_l_prime_array(n):
    """ Compile lp' array (Gusfield theorem 2.2.4) using N array. """
    small_lp = [0] * len(n)
    for i in range(len(n)):
        if n[i] == i+1:  # prefix matching a suffix
            small_lp[len(n)-i-1] = i+1
    for i in range(len(n)-2, -1, -1):  # "smear" them out to the left
        if small_lp[i] == 0:
            small_lp[i] = small_lp[i+1]
    return small_lp


def good_suffix_table(p):
    """ Return tables needed to apply good suffix rule. """
    n = n_array(p)
    lp = big_l_prime_array(p, n)
    return lp, big_l_array(p, lp), small_l_prime_array(n)


def good_suffix_mismatch(i, big_l_prime, small_l_prime):
    """ Given a mismatch at offset i, and given L/L' and l' arrays,
        return amount to shift as determined by good suffix rule. """
    length = len(big_l_prime)
    assert i < length
    if i == length - 1:
        return 0
    i += 1  # i points to leftmost matching position of P
    if big_l_prime[i] > 0:
        return length - big_l_prime[i]
    return length - small_l_prime[i]


def good_suffix_match(small_l_prime):
    """ Given a full match of P to T, return amount to shift as
        determined by good suffix rule. """
    return len(small_l_prime) - small_l_prime[1]


def dense_bad_char_tab(p, amap):
    """ Given pattern string and list with ordered alphabet characters, create
        and return a dense bad character table.  Table is indexed by offset
        then by character. """
    tab = []
    nxt = [0] * len(amap)
    for i in range(0, len(p)):
        c = p[i]
        assert c in amap
        tab.append(nxt[:])
        nxt[amap[c]] = i+1
    return tab


class BoyerMoore(object):
    """ Encapsulates pattern and associated Boyer-Moore preprocessing. """
    
    def __init__(self, p, alphabet='ACGT'):
        self.p = p
        self.alphabet = alphabet
        # Create map from alphabet characters to integers
        self.amap = {}
        for i in range(len(self.alphabet)):
            self.amap[self.alphabet[i]] = i
        # Make bad character rule table
        self.bad_char = dense_bad_char_tab(p, self.amap)
        # Create good suffix rule table
        _, self.big_l, self.small_l_prime = good_suffix_table(p)
    
    def bad_character_rule(self, i, c):
        """ Return # skips given by bad character rule at offset i """
        assert c in self.amap
        ci = self.amap[c]
        assert i > (self.bad_char[i][ci]-1)
        return i - (self.bad_char[i][ci]-1)
    
    def good_suffix_rule(self, i):
        """ Given a mismatch at offset i, return amount to shift
            as determined by (weak) good suffix rule. """
        length = len(self.big_l)
        assert i < length
        if i == length - 1:
            return 0
        i += 1  # i points to leftmost matching position of P
        if self.big_l[i] > 0:
            return length - self.big_l[i]
        return length - self.small_l_prime[i]
    
    def match_skip(self):
        """ Return amount to shift in case where P matches T """
        return len(self.small_l_prime) - self.small_l_prime[1]

In [4]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching """
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences

In [5]:
def approximate_match(p, t, n):
    segment_length = int(round(len(p) / (n+1)))
    all_matches = set()
    for i in range(n+1):
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        p_bm = BoyerMoore(p[start:end], alphabet='ACGT')
        matches = boyer_moore(p[start:end], p_bm, t)
        # Extend matching segments to see if whole p matches
        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start)
    return list(all_matches)

In [13]:
p = 'AACTTG'
t = 'CACTTAATTTG'
print(approximate_match(p, t, 2))

[0, 5]


In [14]:
def editDistRecursive(x, y):
    # This implementation is very slow
    if len(x) == 0:
        return len(y)
    elif len(y) == 0:
        return len(x)
    else:
        distHor = editDistRecursive(x[:-1], y) + 1
        distVer = editDistRecursive(x, y[:-1]) + 1
        if x[-1] == y[-1]:
            distDiag = editDistRecursive(x[:-1], y[:-1])
        else:
            distDiag = editDistRecursive(x[:-1], y[:-1]) + 1
        return min(distHor, distVer, distDiag)

In [15]:
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]

In [16]:
%%time
x = 'shake spea'
y = 'Shakespear'
editDistRecursive(x, y)

Wall time: 8.01 s


In [17]:
%%time
x = 'shake spea'
y = 'Shakespear'
editDistance(x, y)

Wall time: 0 ns


# PATTERN-MATCHING:

In [21]:
class findPattern ():
    """
    This class finds the occurance and position of a given pattern in a given 
    genomic sequence in a file.
    """
    def __init__(self, pattern, filename):
        # initiate parameters
        self.pattern = pattern
        self.filename = filename
        
    def readGenome (self):
        """
        read genomic DNA sequence to a string
        """
        genome = ""
        with open (self.filename, "r") as f:
            for line in f:
                # skip header
                if not line[0] == ">":
                    genome += line.rstrip()
            f.close()
        return genome
        
    
    def reverseComplement (self):
        """
        generate reverse complement sequence for a given dna sequence
        """
        complement = {"A": "T", "C": "G", "T": "A", "G": "C"}
        revComPattern = "" # reversed compliment pattern
        for nt in self.pattern:
            revComPattern = complement[nt] + revComPattern
        
        return revComPattern
        
    def match(self, string1, string2, numOfMismatch):
        """
        return True or False for matching results of two strings under the offset
        of numOfMismatch
        """
        counter = 0
        if len(string1) != len(string2):
            return False
        # loop over string to compare character
        for i in range(len(string1)):
            if string1[i] != string2[i]:
                counter += 1
        if counter > numOfMismatch:
            return False
        return True
        
    def patternIdentifier (self, numOfMismatch):
        """
        find positions of a given pattern and the reversed complement pattern 
        in a given genome
        """
        patternLength = len(self.pattern)
        genome = self.readGenome()
        revComPattern = self.reverseComplement()
        occurances = []
        
        for i in range (patternLength): # loop over pattern index
            # loop over genome patterns
            for j in range (i, len(genome), patternLength): 
                genomeMotif = genome[j: j+patternLength]
                # compare genomic motif and patterns
                if numOfMismatch == 0:
                    if (self.match(genomeMotif, self.pattern, 0) or \
                        self.match(genomeMotif, revComPattern, 0))\
                       and j not in occurances: # avoid duplicate records
                        occurances.append(j)
                else:
                    if self.match(genomeMotif, self.pattern, numOfMismatch)\
                       and j not in occurances: # avoid duplicate records:
                        occurances.append(j)
        return occurances
    
class checkQuality ():
    """
    The checkQuality class exams quality of sequencing for each cycle
    """
    def __init__ (self, filename):
        self.filename = filename
    
    def readFastq(self):
        """
        read dna sequence and quality base from a fastq sequencing file to lists
        """
        with open (self.filename, "r") as f:
             sequences = []
             qualities = []
             while True:
                 f.readline() # skip name line
                 seq = f.readline().rstrip() # read sequence line
                 f.readline() # skip strand line
                 qual = f.readline().rstrip() # read quality line
                 if len(seq) == 0: #finish read
                     break
                 # add seqence and quality information to list
                 sequences.append(seq)
                 qualities.append(qual)
             f.close()
        return sequences, qualities
        
    def phre33ToQ (self, qualString):
        """
        transform quality string to quality base score
        """
        qScore = []
        for qual in qualString:
            qScore.append(ord(qual) - 33)
        return qScore
    def findPoorQuality(self):
        """
        find the index of poorest q score in each sequencing
        """
        _, qualities = self.readFastq()
        lowestQScoreIndex = []
        for qualString in qualities:
            qScore = self.phre33ToQ(qualString)
            lowestQScoreIndex.append(qScore.index(min(qScore)))
        return lowestQScoreIndex
        
    def countPoorQuality(self):    
        """
        count number of poorest q score in each cycle
        """
        import collections
        return collections.Counter(self.findPoorQuality())
        
    def plotHist(self):
        """
         show the distribution of poorest q score
        """
        import matplotlib.pyplot as plt
        data = self.countPoorQuality()
        plt.bar(data.keys(), data.values())
        plt.show()


if __name__ == "__main__":
    #Test
    filename = "CERVICALDNA.fasta"
    pattern = "ATTA"
    patterns = findPattern(pattern,filename)
    print("Test dataset results - Occurences and leftmost offset: ")
    print(len(patterns.patternIdentifier(0)), min(patterns.patternIdentifier(0)), "\n")
    
    #Questions 1-6
    filename1 = "CERVICALDNA.fasta"
    #Q1: How many times does AGGT or its reverse complement (ACCT) occur in the 
    #Cervical Cancer genome? E.g. if AGGT occurs 10 times and ACCT occurs 12 times, 
    #you should report 22.
    pattern = "AGGT"
    patterns = findPattern(pattern,filename1)
    print("Q1: The 'AGGT' or 'ACCT' occurs %d times \n" \
           %len(patterns.patternIdentifier(0)))
    
    #Q2: How many times does TTAA or its reverse complement occur in the Cervical Cancer genome? 
    #Hint: TTAA and its reverse complement are equal, so remember 
    #not to double count.    
    pattern = "TTAA"
    patterns = findPattern(pattern,filename1)
    print("Q2: The 'TTAA' occurs %d times \n" \
           %len(patterns.patternIdentifier(0)))
    
    #Q3: What is the offset of the leftmost occurrence of GGCGCGC or its reverse
    #complement in the Cervical Cancer genome? E.g. if the leftmost occurrence of 
    #GGCGCGC is at offset 40 (0-based) and the leftmost occurrence of its reverse 
    #complement GGCGCGC is at offset 29, then report 29.
    pattern = "GGCGCGC"
    patterns = findPattern(pattern,filename1)
    print("Q3: The offset of the leftmost occurrence of GGCGCGC is %d \n" \
           %min(patterns.patternIdentifier(0)))
    
    #Q4: What is the offset of the leftmost occurrence of GTTTGC or its reverse 
    #complement in the Cervical Cancer genome?
    pattern = "GTTTGC"
    patterns = findPattern(pattern,filename1)
    print("Q4: The offset of the leftmost occurrence of GTTTGC is %d \n" \
           %min(patterns.patternIdentifier(0)))
           
    #Q5: How many times does TTCAAGCC occur in the Cervical Cancer genome when 
    #allowing up to 2 mismatches?    
    pattern = "TTCAAGCC"
    patterns = findPattern(pattern,filename1)
    print("Q5: The 'TTCAAGCC' occurs %d times with mismatch less than 2 \n" \
           %len(patterns.patternIdentifier(2)))
       
    #Q6: What is the offset of the leftmost occurrence of AGGAGGTT in the 
    #Cervical Cancer genome when allowing up to 2 mismatches?   
    pattern = "AGGAGGTT"
    patterns = findPattern(pattern,filename1)
    print("Q6: The offset of the leftmost occurrence of AGGAGGTT with mismatch less than 2 is %d \n" \
          %min(patterns.patternIdentifier(2)))      
      
    #Q7: Report which sequencing cycle has the problem.    
    filename2 = "CERVICALDNA.fasta"
    qualities = checkQuality(filename2)

Test dataset results - Occurences and leftmost offset: 
4 258 

Q1: The 'AGGT' or 'ACCT' occurs 2 times 

Q2: The 'TTAA' occurs 1 times 

Q3: The offset of the leftmost occurrence of GGCGCGC is 71 

Q4: The offset of the leftmost occurrence of GTTTGC is 320 

Q5: The 'TTCAAGCC' occurs 1 times with mismatch less than 2 

Q6: The offset of the leftmost occurrence of AGGAGGTT with mismatch less than 2 is 4 



In [25]:
from itertools import permutations

class findPatternV3 ():
    """
    This class finds the edit distance of a given pattern in a given genomic 
    sequence and constructs overlap graphs.
    """
    def __init__(self, filename, pattern = False):
        # initiate parameters
        self.pattern = pattern
        self.filename = filename
        
    def readGenome (self):
        """
        read genomic DNA sequence to a string
        """
        genome = ""
        with open (self.filename, "r") as f:
            for line in f:
                # skip header
                if not line[0] == ">":
                    genome += line.rstrip()
            f.close()
        return genome
        
    def readFastq(self):
        """
        read dna sequence and quality base from a fastq sequencing file to lists
        """
        with open (self.filename, "r") as f:
             sequences = []
             qualities = []
             while True:
                 f.readline() # skip name line
                 seq = f.readline().rstrip() # read sequence line
                 f.readline() # skip strand line
                 qual = f.readline().rstrip() # read quality line
                 if len(seq) == 0: #finish read
                     break
                 # add seqence and quality information to list
                 sequences.append(seq)
                 qualities.append(qual)
             f.close()
        return sequences, qualities
        
    def editDistance(self):
        """
        Implement dynamic algorithm to calculate edit distance between a given
        pattern and a given genome
        """
        pattern = self.pattern
        genome = self.readGenome()
        pattern_length = len(pattern) + 1
        genome_length = len(genome) + 1
        #generate matrix
        matrix = [[0]*genome_length for i in range(pattern_length)]
        # initiate the first column
        for i in range(pattern_length):
            matrix[i][0] = i
        for i in range(1, pattern_length):
            for j in range(1, genome_length):
                dist_hor = matrix[i][j-1] + 1
                dist_vel = matrix[i-1][j] + 1
                dist_diag = matrix[i-1][j-1] + 1 if pattern[i-1] != genome[j-1]\
                            else matrix[i-1][j-1]
                matrix[i][j] = min(dist_hor, dist_vel, dist_diag)
        return min(matrix[-1])                
            
    def phraseReads(self, k_mer):
        """
        construct the prefix and suffix of a read to a dictornary with read as
        key and pre-,suffix as values
        """
        reads, _ = self.readFastq()
        reads_dict = {}
        for read in reads:
            for i in range(len(read) - k_mer + 1):
                substring = read[i:i+k_mer]
                if substring not in reads_dict:
                    reads_dict[substring] = set([read])
                else:
                    reads_dict[substring].add(read)
        return reads_dict
    def overlap(self, read1, read2, k_mer):
        """
        find overlaped leftmost offset
        """
        start = 0
        while True:
            start = read1.find(read2[:k_mer], start)
            if start == -1:
                return 0 # without overlap
            if read2.startswith(read1[start:]):
                return len(read1) - start
            start += 1
            
    def overlapGraph(self, k_mer):
        """
        construct graph with key as a read (node) and values as all other 
        reads overlapped with the previous read (node)
        """
        reads_dict = self.phraseReads(k_mer)
        reads, _= self.readFastq()
        graph = {}
        for read1 in reads:
            k_mer_string = read1[len(read1) - k_mer:]
            if k_mer_string in reads_dict:
                edges = set([])
                reads_set = reads_dict[k_mer_string]
                for read2 in reads_set:
                    if read1 != read2: #skip self comparison
                        offset = self.overlap(read1, read2, k_mer)           
                        if offset > 0: # skip non-overlapped pairs
                            edges.add(read2) #add overlapped reads to be values
                            graph[read1] = edges           
        return graph
                        
    def naive_overlap_map(self, k_mer):
        """
        construct graph with key as a pair of reads with overlap and values as
        the leftmost offset of the overlap
        """
        graph = {}
        reads, _ = self.readFastq()
        for read1, read2 in permutations(reads, 2):
            #skip non-overlapped reads
            if read1[len(read1) - k_mer:] in read2:
                offset = self.overlap(read1, read2, k_mer)
                    # check if reads[i] overlapped with reads[j]
                if offset != 0:
                    graph[(read1,read2)] = offset  
        return graph

if __name__ == "__main__":
    
    #Q1: What is the edit distance of the best match between pattern 
    #ATTTATCCGCCGATTCC and the excerpt of human chromosome 1? 
    #(Don't consider reverse complements.)
    pattern = "ATTTATCCGCTTCC"
    filename = "CERVICALDNA.fasta"
    patterns = findPatternV3 (filename, pattern)
    edit_dist = patterns.editDistance()
    print("Q1: the edit distance of the best match between pattern and the genome is %d\n"\
           %edit_dist)
           
    #Q2: What is the edit distance of the best match between pattern 
    #GCCGATTCCGGGCG and the excerpt of human chromosome 1? 
    #(Don't consider reverse complements.)
    pattern = "GCCGATTCGGCG"
    filename = "CERVICALDNA.fasta"
    patterns = findPatternV3 (filename, pattern)
    edit_dist = patterns.editDistance()
    print("Q2: the edit distance of the best match between pattern and the genome is %d\n"\
           %edit_dist)
    
    #Q3: Picture the overlap graph corresponding to the overlaps just calculated. 
    # How many edges are in the graph? In other words, how many distinct pairs 
    # of reads overlap?
    #Q4: Picture the overlap graph corresponding to the overlaps computed for 
    #the previous question. How many nodes in this graph have at least one 
    #outgoing edge? (In other words, how many reads have a suffix involved in 
    #an overlap?)
    import time
    t1 = time.time()
    filename = "ConcatDNA.fastq"
    patterns = findPatternV3 (filename)
    k_mer = 30
    graph = patterns.naive_overlap_map(k_mer)
    t2 = time.time()
    print("Running time for naive overlap mapping: %d sec\n"%(t2 - t1))
    
    reads = patterns.phraseReads(k_mer)
    t3 = time.time()
    print("Running time for phrase reads: %d sec\n"%(t3 - t2))
    graph = patterns.overlapGraph(k_mer)
    t4 = time.time()
    print("Running time for optimized algorithm: %d sec\n"%(t4 - t2))
    numberOfNodes = len(graph)
    numberOfEdges = sum([len(edges) for edges in graph.values()])
    print("Q3: the total edges are %d\n"%numberOfEdges)
    print("Q4: the total nodes are %d"%numberOfNodes)

Q1: the edit distance of the best match between pattern and the genome is 3

Q2: the edit distance of the best match between pattern and the genome is 2

Running time for naive overlap mapping: 4 sec

Running time for phrase reads: 0 sec

Running time for optimized algorithm: 0 sec

Q3: the total edges are 3

Q4: the total nodes are 2
